In [ ]:
pip install pulp

In [ ]:
import numpy as np

In [ ]:
loc = np.array([[54.29, 54.14, 54.21, 54.7, 55.22],
                [54.77, 54.57, 54.39, 54.24, 54.1]])

In [ ]:
t_f = np.array([[1.67, 1.67, 1.88, 2.50, 3.13],
               [3.33, 3.33, 3.33, 3.33, 3.33]])

In [ ]:
ratio = [
    1, 19.4, 9.5, 3.5, 1.1, 11, 2, 3.8, 8.2, 6.3, 53, 4.1, 30, 26.5, 24.7, 36,
    7, 6.5, 28, 127, 4.5, 6, 20, 15.5, 14.5, 3.7, 4.3, 5, 3.1, 5.6, 9.2, 89,
    33, 23, 58, 1.6, 10, 9, 2.5, 5.2, 49, 62, 4, 69.5, 22, 294, 21, 13.9, 18.3,
    38.7, 78, 10.5, 7.8, 105, 13, 17.1, 7, 42, 3, 6, 12, 46, 2.1
]


In [ ]:
p = [132, 97, 124, 52, 299, 22, 106, 161, 90, 75, 53, 65, 90, 106, 74, 36, 7, 39, 28, 127, 63, 6, 60, 31, 29, 55, 30, 5, 65, 39, 83, 89, 33, 23, 58, 181, 10, 9, 33, 67, 49, 62, 4, 139, 22, 294, 21, 139, 55, 116, 78, 42, 39, 105, 52, 137, 7, 42, 3, 72, 12, 46, 72]

In [ ]:
import pulp

problem = pulp.LpProblem("Binary_IP_Problem", pulp.LpMinimize)

product = [i for i in range(63)]
rows = [0,1,2,3,4]
columns = [0,1]
d_val= [
    132, 5, 13, 15, 267, 2, 54, 42, 11, 12, 1, 16, 3, 4, 3, 1, 1, 6, 1, 1,
    14, 1, 3, 2, 2, 15, 7, 1, 21, 7, 9, 1, 1, 1, 1, 112, 1, 1, 13, 13, 1,
    1, 1, 2, 1, 1, 1, 10, 3, 3, 1, 4, 5, 1, 4, 8, 1, 1, 1, 12, 1, 1, 34
]


x = pulp.LpVariable.dicts("x", ((k, i, j) for k in product for i in rows for j in columns), cat='Binary')

problem += pulp.lpSum(
    sum(
        sum(
            (loc[j][i] * x[(k, i, j)] * d_val[k]) / ratio[k]
            for j in columns
        )
        for i in rows
    )
    for k in product
) +pulp.lpSum(
    sum(
        sum(
            (4*t_f[j][i] * x[(k, i, j)] * p[k])
            for j in columns
        )
        for i in rows
    )
    for k in product
) + pulp.lpSum(
    4 * 12.02 * (1 - x[(k, i, j)]) * p[k]
    for k in product for i in rows for j in columns
)



for k in product:
    problem += pulp.lpSum(x[(k, i, j)] for i in rows for j in columns) <= 1, f"Constraint_for_k_{k}"

for i in rows:
  for j in columns:
    problem += pulp.lpSum(x[(k, i, j)] for k in product) == 1, f"constraint_for_storage_{i}_{j}"


# Solve the problem
problem.solve()

# Output results
print(f"Status: {pulp.LpStatus[problem.status]}")
print(f"Total Value of selected items: {pulp.value(problem.objective)}")
selected_items = [k+1 for k, i, j in x if pulp.value(x[(k, i, j)]) == 1]
print(f"Selected items: {selected_items}")

Status: Optimal
Total Value of selected items: 2042309.4081543537
Selected items: [3, 8, 14, 20, 44, 46, 48, 50, 54, 56]


In [ ]:
solution_list = []
for k, i, j in x:
  if pulp.value(x[(k, i, j)]) == 1:
    solution_list.append(x[(k, i, j)])
    print(x[(k, i, j)])

x_(2,_4,_1)
x_(7,_1,_0)
x_(13,_3,_1)
x_(19,_0,_1)
x_(43,_3,_0)
x_(45,_0,_0)
x_(47,_2,_0)
x_(49,_2,_1)
x_(53,_1,_1)
x_(55,_4,_0)


In [ ]:
matrix = np.zeros((2,5))
for k, i, j in x:
    if pulp.value(x[(k, i, j)]) == 1:
        matrix[j][i] = k + 1

print(matrix)

[[46.  8. 48. 44. 56.]
 [20. 54. 50. 14.  3.]]
